# 1. Compute angles and energies from coordinates and save them with Semg and labels

In [1]:
import scipy.io
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing

# id of participants
train_participant_num = ["C56D","C93D","C382D","C382N","C544D","C709N","C788N","P113D",
                         "P113N","P191D","P191N","P299D","P299N","P300D","P336D","P492D",
                         "P492N","P531N","P699D","P699N","P890N","P921D","P921N"]
valid_participant_num = ["C67D","C202D","C202N","C256D","C256N","P54D","P54N","P342D",
                         "P342N","P487D","P487N","P649N"]

# joints conatined in each angle
angle1 = [22,1,4]
angle2 = [22,1,7]
angle3 = [8,1,3]
angle4 = [8,1,6]
angle5 = [2,3,4]
angle6 = [5,6,7]
angle7 = [11,12,13]
angle8 = [16,17,18]
angle9 = [20,10,11]
angle10 = [20,15,16]
angle11 = [12,10,2]
angle12 = [17,15,5]
angle13 = [8,20,22]
angles = [angle1, angle2, angle3, angle4, angle5, angle6, angle7, angle8, angle9, angle10, angle11, angle12 ,angle13]

energy_cols = ['energy1', 'energy2', 'energy3', 'energy4', 'energy5', 'energy6', 'energy7', 
               'energy8', 'energy9', 'energy10', 'energy11', 'energy12', 'energy13']

# compute the angle
def compute_angle(row, angle):
    xA, yA, zA = row[angle[0]-1], row[22 + angle[0]-1], row[44 + angle[0]-1]
    xB, yB, zB = row[angle[1]-1], row[22 + angle[1]-1], row[44 + angle[1]-1]
    xC, yC, zC = row[angle[2]-1], row[22 + angle[2]-1], row[44 + angle[2]-1]
    AB = (xB-xA, yB-yA, zB-zA)
    BC = (xC-xB, yC-yB, zC-zB)

    lenAB = math.sqrt(AB[0]**2 + AB[1]**2 + AB[2]**2)
    lenBC = math.sqrt(BC[0]**2 + BC[1]**2 + BC[2]**2)

    dotABBC = AB[0]*BC[0] + AB[1]*BC[1] + AB[2]*BC[2]

    theta = math.acos(dotABBC / (lenAB * lenBC))
    return theta

# get the new dataset including 13 angles, 13 energy and 4 semg data
def get_new_dataset(df, energy_scaling = False):
    df_angle = pd.DataFrame(columns=['angle1', 'angle2', 'angle3', 'angle4', 'angle5', 'angle6', 'angle7', 
                                 'angle8', 'angle9', 'angle10', 'angle11', 'angle12' ,'angle13'])
    
#     compute angle columns
    for i in range(len(angles)):
        my_angle = angles[i]
        result = df.apply(compute_angle, angle=my_angle, axis=1)
        df_angle['angle'+ str(i+1)] = result
        
#     compute energy columns
    df_angle[energy_cols] = df_angle.diff().fillna(0).apply(lambda x: np.square(x * 60))
    
#     scaling the energy columns
    if energy_scaling:
        energy_arr = df_angle[energy_cols].values
        min_max_scaler = preprocessing.MinMaxScaler()
        energy_scaled = min_max_scaler.fit_transform(energy_arr)
        df_angle[energy_cols] = pd.DataFrame(energy_scaled)
        
#     get other columns
    select_cols=df.columns[66:70]
    df_angle[['semg1', 'semg2', 'semg3', 'semg4']] = df[select_cols]
    df_angle[['label1', 'label2', 'label3', 'label4', 'label5', 'label6']] = df[df.columns[72:78]]
    return df_angle

In [2]:
for num in train_participant_num:
    mat = scipy.io.loadmat('../CoordinateData/train/' + num + '.mat') 
    df = pd.DataFrame(mat['data'])
    df_angle = get_new_dataset(df)
    data_angle = df_angle.to_numpy()
    mat_angle = {'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64',
                 '__version__': '1.0', '__globals__': [], 'data':data_angle}
    scipy.io.savemat('../AngleData/train/' + num + '.mat', mat_angle)
    
for num in valid_participant_num:
    mat = scipy.io.loadmat('../CoordinateData/validation/' + num + '.mat') 
    df = pd.DataFrame(mat['data'])
    df_angle = get_new_dataset(df)
    data_angle = df_angle.to_numpy()
    mat_angle = {'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64',
                 '__version__': '1.0', '__globals__': [], 'data':data_angle}
    scipy.io.savemat('../AngleData/validation/' + num + '.mat', mat_angle)

# 2. Add angle_energy to the orginal dataset

In [3]:
for num in train_participant_num:
    mat_coor = scipy.io.loadmat('../CoordinateData/train/' + num + '.mat') 
    df_coor = pd.DataFrame(mat_coor['data'])
    mat_angle = scipy.io.loadmat('../AngleData/train/' + num + '.mat') 
    df_angle = pd.DataFrame(mat_angle['data'])
    df_all = pd.concat([df_coor, df_angle[df_angle.columns[0:26]]], axis=1)
    data_all = df_all.to_numpy()
    mat_all = {'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64',
                 '__version__': '1.0', '__globals__': [], 'data':data_all}
    scipy.io.savemat('../AllData/train/' + num + '.mat', mat_all)
    
for num in valid_participant_num:
    mat_coor = scipy.io.loadmat('../CoordinateData/validation/' + num + '.mat') 
    df_coor = pd.DataFrame(mat_coor['data'])
    mat_angle = scipy.io.loadmat('../AngleData/validation/' + num + '.mat') 
    df_angle = pd.DataFrame(mat_angle['data'])
    df_all = pd.concat([df_coor, df_angle[df_angle.columns[0:26]]], axis=1)
    data_all = df_all.to_numpy()
    mat_all = {'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64',
                 '__version__': '1.0', '__globals__': [], 'data':data_all}
    scipy.io.savemat('../AllData/validation/' + num + '.mat', mat_all)